# Ejemplo 1 de Agente Dinámico: Responder en el Idioma del Usuario

## Objetivo
* Hacer que el agente responda en español cuando habla con hispanohablantes, y en inglés para los anglohablantes.

## El código

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from dataclasses import dataclass
from langchain.agents.middleware import dynamic_prompt, ModelRequest

@dataclass
class LanguageContext:
    user_language: str = "English"

@dynamic_prompt
def user_language_prompt(request: ModelRequest) -> str:
    """Generate system prompt based on user role."""
    user_language = request.runtime.context.user_language
    base_prompt = "You are a helpful assistant."

    if user_language != "English":
        return f"{base_prompt} only respond in {user_language}."
    elif user_language == "English":
        return base_prompt

from langchain.agents import create_agent

agent = create_agent(
    model="gpt-4o-mini",
    context_schema=LanguageContext,
    middleware=[user_language_prompt]
)

from langchain.messages import HumanMessage

response = agent.invoke(
    {"message": [HumanMessage(content="Hola, ¿cómo estás?")]},
    context=LanguageContext(user_language="Spanish")
)

print(response["messages"][-1].content)

## Expliquemos el código anterior en términos sencillos

A continuación encontrarás una **explicación línea por línea para principiantes** del código.

---

## Qué hace este código (visión general)

Este código crea un **agente dinámico de LangChain** que:

* Conoce el idioma preferido del usuario (inglés, español, etc.)
* **Cambia automáticamente el prompt del sistema**
* Fuerza a la IA a **responder únicamente en ese idioma**

Piénsalo como:

> "Antes de que la IA responda, comprueba en qué idioma quiere hablar el usuario y ajusta las instrucciones en consecuencia."


---

## 1. Importaciones y configuración

```python
from dataclasses import dataclass
from langchain.agents.middleware import dynamic_prompt, ModelRequest
```

#### ¿Qué está pasando aquí?

* `dataclass` (de Python) se utiliza para definir **contenedores de datos simples**
* `dynamic_prompt` es un **decorador de LangChain** que marca una función como algo que puede **modificar prompts en tiempo de ejecución**
* `ModelRequest` es un objeto que contiene **todo sobre la petición actual**, incluyendo:

  * el modelo que se está utilizando
  * mensajes
  * contexto de ejecución (como la configuración del usuario)

---

## 2. Definir el contexto (estado compartido)

```python
@dataclass
class LanguageContext:
    user_language: str = "English"
```

#### ¿Qué está pasando aquí?

* Esto define un **objeto de contexto** que viaja con cada petición
* Almacena **información adicional** que NO forma parte del mensaje del usuario
* En este caso, almacenamos:

  * `user_language`: el idioma que prefiere el usuario

#### Por qué esto es importante

En lugar de adivinar el idioma del mensaje, nosotros:

* Le decimos explícitamente al agente qué idioma utilizar
* Mantenemos esta lógica limpia y predecible

Piensa en `LanguageContext` como:

> "Configuración adicional sobre el usuario que la IA debería conocer."

---

## 3. Crear un prompt de sistema dinámico

```python
@dynamic_prompt
def user_language_prompt(request: ModelRequest) -> str:
    """Generate system prompt based on user language."""
```

#### ¿Qué está pasando?

* `@dynamic_prompt` le dice a LangChain:

  > "Esta función generará un **prompt de sistema** de forma dinámica para cada petición."
* LangChain llamará automáticamente a esta función **antes** de que se ejecute el modelo

---

#### Acceder al contexto

```python
    user_language = request.runtime.context.user_language
```

* `request` es el objeto de petición completo
* `request.runtime.context` contiene el objeto de contexto que pasamos
* `user_language` se lee de `LanguageContext`

En español sencillo:

> "Busca el idioma preferido del usuario."

---

#### Instrucción base del sistema

```python
    base_prompt = "You are a helpful assistant."
```

* Esta es la instrucción por defecto del sistema
* Todo lo demás se construye sobre esto

---

#### Lógica condicional

```python
    if user_language != "English":
        return f"{base_prompt} only respond in {user_language}."
    elif user_language == "English":
        return base_prompt
```

#### ¿Qué está pasando?

* Si el idioma del usuario **no es inglés**:

  * Añadimos una instrucción estricta diciéndole al modelo que responda únicamente en ese idioma
* Si el idioma **es inglés**:

  * Mantenemos el prompt simple

Ejemplos de salida:

* Español →
  `"You are a helpful assistant. only respond in Spanish."`
* Inglés →
  `"You are a helpful assistant."`

Esta cadena devuelta se convierte en el **prompt del sistema** enviado al modelo.

---

## 4. Crear el agente

```python
from langchain.agents import create_agent
```

Esto importa la función auxiliar de LangChain para construir agentes.

---

```python
agent = create_agent(
    model="gpt-4o-mini",
    context_schema=LanguageContext,
    middleware=[user_language_prompt]
)
```

#### ¿Qué está pasando aquí?

* `model="gpt-4o-mini"`
  → Selecciona el LLM a utilizar

* `context_schema=LanguageContext`
  → Le dice a LangChain:

  > "Cada petición puede incluir un objeto `LanguageContext`"

* `middleware=[user_language_prompt]`
  → Registra nuestra función de prompt dinámico
  → LangChain la ejecutará **antes de cada llamada al modelo**

Piensa en el middleware como:

> "Código que se ejecuta *entre* la entrada del usuario y la respuesta de la IA."

---

## 5. Enviar un mensaje al agente

```python
from langchain.messages import HumanMessage
```

* `HumanMessage` representa un mensaje del usuario en el formato de mensajes de LangChain

---

```python
response = agent.invoke(
    {"message": [HumanMessage(content="Hola, ¿cómo estás?")]},
    context=LanguageContext(user_language="Spanish")
)
```

#### ¿Qué está pasando paso a paso?

1. El usuario envía un mensaje en español
2. Pasamos explícitamente:

   ```python
   context=LanguageContext(user_language="Spanish")
   ```
3. LangChain:

   * Almacena este contexto en `request.runtime.context`
   * Llama a `user_language_prompt`
   * Construye el prompt del sistema:

     ```
     You are a helpful assistant. only respond in Spanish.
     ```
4. El modelo recibe:

   * Prompt del sistema (del middleware)
   * Mensaje del usuario
5. El modelo responde en español

---

## 6. Imprimir la respuesta final

```python
print(response["messages"][-1].content)
```

#### ¿Qué está pasando?

* `response["messages"]` es una lista de todos los mensajes (sistema, usuario, asistente)
* `[-1]` obtiene el **último mensaje**, que es la respuesta de la IA
* `.content` extrae el texto

---

## Modelo mental final (¡importante!)

Piensa en este flujo:

```
Mensaje del usuario
   ↓
Contexto (LanguageContext)
   ↓
Middleware de prompt dinámico
   ↓
Se genera el prompt del sistema
   ↓
Se llama al LLM
   ↓
Respuesta
```

---

# Por qué este patrón es poderoso

* ✅ Separación clara de responsabilidades
* ✅ Sin trucos de prompt en los mensajes del usuario
* ✅ Comportamiento basado en contexto
* ✅ Fácil de extender (tono, rol, permisos, etc.)

## Cómo ejecutar este código desde Visual Studio Code
* Abre el Terminal.
* Asegúrate de estar en la carpeta del proyecto.
* Asegúrate de tener activado el entorno de poetry.
* Introduce y ejecuta el siguiente comando:
    * `python 015-dyn-agent-customize-language.py`